#WomboCombo: Optimizing Team Composition in League of Legends
### by Data of Legends (Sean Cha, Jaemin Cheun, Kenny Lei, Andrew Paek) 
###Assigned TF: Léandra King

<img src="http://36646d87786feafc0611-0338bbbce19fc98919c6293def4c5554.r0.cf1.rackcdn.com/images/FiGZ9r3D3E82.878x0.Z-Z96KYq.jpg" width=610 height=347/>

#Table of Contents

* [WomboCombo: Optimizing Team Composition in League of Legends](#WomboCombo: Optimizing Team Composition in League of Legends)
	* [Introduction](#Introduction)
		* [Different Roles](#Different Roles)
        * [Objectives](#Objectives)
	* [Part I: Data Retrieval & Cleaning](#Part-I-Data-Retrieval-&-Cleaning)
		* [League of Legends API](#LoL-API)
        * [Pulling and Saving JSON Files](#JSON-Files)
		* [Pre-processing Data](#pre-processing)
		* [Feature Selection](#feature-selection)
        * [Parsing JSON into a csv](#parsing)
    * [Part II: Exploratory Data Analysis (EDA)](#EDA)
		* [Rank Tier and Region Distribution](#tier-region-dist)
		* [Most Popular Champions](#popular_champ)
        * [Win/Loss Rate Based on Champion](#win_loss_rate)
        * [KDA by Champion](#KDA)
        * [Champions by Stat](#champion-stat)
    * [Part III: Clustering Algorithms](#clustering)
		* [Making baseline predictions](#Making-baseline-predictions)
			* [1.3 Make the prediction from the `baseline` model](#1.3-Make-the-prediction-from-the-baseline-model)        

##Introduction <a id='Introduction'></a>

League of Legends (LoL) is currently the most popular game in the world. LoL is a fast-paced, competitive online game that blends the speed and intensity of an RTS with RPG elements. Two teams of 5 powerful champions, each with a unique design and playstyle, battle head-to-head across multiple battlefields and game modes. With an ever-expanding roster of champions, frequent updates and a thriving tournament scene, League of Legends offers endless replayability for players of every skill level.

The main objective of the game is to knock down the enemies' towers using the champions, while the enemy team is trying to do the same to ours. Towers are difficult to destroy with the enemy defending them, so most of the game revolves around killing enemy champions in order to buy the necessary time to bring down enemy towers.

Team composition is extremely important in the game. Each champion out of the roster of 128 (and growing!) carries its own unique abilities, utility, and play style that it can contribute to the team. Simply put, a team of 5 champions that share similar roles in the game would be very ineffective against a well-balanced team. Let's look at some examples.

### Different Roles <a id='Different Roles'></a>
This is LeBlanc. Her primary role in the game is an assassin. She has an extremely low physical damage power (red bar) and low health (green bar). However, she has a HUGE ability power (blue bar), allowing her to use quick burst of ability power to kill enemies in a split second. She is extremely effective in assassinating enemy champions that are roaming around the map by themselves. Since her abilities have a very long cool-down (time for her to be able to cast spells again), she is in deep trouble if she cannot kill the enemy champion the first time around. In a team fight, LeBlanc focuses on killing high priority enemy champions. This is an old video from a few seasons back, but it really demonstrates what LeBlanc is capable of: https://www.youtube.com/watch?v=lsBJEvwi66k.
<img src=http://i.imgur.com/G5jbBpq.png>

This is Malphite. His primary role is a tank. He has high health and defensive statistics, allowing him to last a long time in a team fight. Although he cannot quickly kill enemy champions the way LeBlanc does, he is able to follow around and harrass/interrupt high-priority enemy champions in their attempts to kill ours. He is also an "utility" champion, meaning that he has various "crowd-controlling" abilities that slow down and knocks up enemy champions, making it easier for our champions to target them. This is an example of Malphite's specialty in initiating a team fight and tankiness: https://youtu.be/pE3aLnoB7oA?t=17m53s.
<img src=http://i.imgur.com/POWWt9u.png>

These are only 2 of the 128 champions that players can choose to play. As you can see, LeBlanc and Malphite have very distinctly different roles in the game. It would be a disaster for a team to have 5 "tank" champions, since tanky champions, despite their resilience, cannot deal enough damage to kill enemy champions. A team of only assassins will also have no luck in winning; without a reliable tanker to take the frontline and disrupt the enemy, the LeBlancs will quickly "evaporate to death."

This is where team composition comes in. A team that has both a Malphite and a LeBlanc will be extremely effective. While Malphite slows down enemies and takes all the damages, LeBlanc can focus on dealing massive magic damage to the enemy champions. Other notable roles include that of a fighter (a champion that can deal consistent damage over time while taking moderate damage), a support (a champion specializing in crowd control abilities to disable enemy champions and healing to protect our champions), a marksman (a range-attack champion who is extremely vulnerable but can deal a massive amount of damage from a distance), etc.

Certain set of champions complement each other really well, thus the coinage of the term, Wombo Combo. To look at some examples of Wombo Combos, please see: https://www.youtube.com/watch?v=TW6bdxnFcjs

###Objectives <a id='Objectives'></a>

We hope to gain insight on optimal team composition by analyzing match data.

First, we use Riot's League of Legends API (https://developer.riotgames.com/docs/rate-limiting) to download the necessary data. Then, we preprocess and clean the raw JSON data to a workable structure.

Next, we conduct Exploratory Data Analysis (EDA) to answer some basic questions regarding our data set. By visualizing and playing around with the data, we gain additional insight that will be helpful in later parts.

Then, we ... (WRITE ABOUT ALGORITHMS HERE)

In [1]:
import os
import json
import requests
import time
import datetime
import types
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

##Part I: Data Retrieval & Cleaning<a id='Part-I-Data-Retrieval-&-Cleaning'></a>

### League of Legends API <a id='LoL-API'></a>

Rate Limiting: The API allows one user to make 10 requests per 10 seconds and 500 requests per 10 minutes.

For documentation, see https://developer.riotgames.com/api/methods.

From the riot games API page,  
<b>"The rate limit for development keys is by design very limited:   
10 requests every 10 seconds  
                                                                500 requests every 10 minutes" </b>

However,

<b>"Note that rate limits are enforced per region. For example, with the above rate limit, you could make 10 requests every 10 seconds to both NA and EUW endpoints simultaneously."</b>

If we try to only stick to North American server data, we will have to wait 10 seconds for every 10 requests. This would be very unfortunately inefficient. However, if we pull data from all of LoL's 10 global regions, we can continuously pull data without the rate limit's hinderance. When we reach the limit, we'll just jump regions!

There may be slight variations in popular game styles from region to region. But keep in mind that LoL is a same game no matter where in the world you are. Whenever there is a champion update or an item patch, the entire world's LoL platform is updated simultaneously.

In [2]:
# API KEY HERE
KEY = 'b00c992b-6fc1-4795-abcc-7db85c1b72fa' # API Key associated with Sean's LoL account (ch920425)

We define a function that makes a request for a match record by its matchID.

In [3]:
# MATCH-v2.2
REGION_ENDPOINT = "https://{0}.api.pvp.net/api/lol/{0}/"

# FUNCTION TO PULL MATCH DATA
def get_match(REGION, matchId, includeTimeline):
    """
    Retrieve match by match ID.
    """
    return requests.get(
        (REGION_ENDPOINT + "v2.2/match/{1}?"
         "api_key={2}&includeTimeline={3}").
        format(REGION, matchId, KEY, includeTimeline))

We establish constants that allow us to check whether we are within the API's rate limit. 

We define several functions that allow us to filter out the data we want.

There are different "game modes" and "queue types" in League of Legends. We only want to look at the most popular type people play, which are "classic" 5v5 match on a map called Summoner's Rift. We also want to only look at "Ranked" games, which affect the player's scores and ranking within the gaming ecosystem. In Ranked games, people are motivated to put in their best efforts to advance to higher tier systems. Simply put, classic ranked game data will manifest champion selections and play styles that players themselves would find optimal. We want to capture this optimally conventionally play style and find patterns.

However...

API Limitations: The API does not allow us to specify what type of game mode or queue type we want to pull. There is no real distinguishable pattern within matchID's neither. We will have to pick a starting point (an arbitrary matchID), increment the ID by 1, pull the data, check to see whether it satisfies our requirements, and then decide to store it.

This process does take a long time. We have run the scripts to selectively pull data this way for days and nights on multiple machines.

In [4]:
# Constants
STATUS_OK = 200                  # Shows that we have successfully pulled data
STATUS_RATE_LIMIT_EXCEEDED = 429 # Tells us that we have to wait 10 seconds before requesting again

# Region Servers from which to retrieve data
# North America, Korea, Europe Nordic and East, Europe West, Latin America 
REGION_list = ['na',     # North America
               'kr',     # Korea
               'eune',   # Europe Nordic and East
               'euw',    # Europe West
               'lan',    # Latin America North
               'las',    # Latin America South
               'oce',    # Oceania
               'ru',     # Russia
               'br',     # Brazil
               'tr']     # Turkey

# Only interested in 5v5 RANKED games
valid_rank_games = ['RANKED_SOLO_5x5',     # Ranked Solo 5v5 games
                    'RANKED_PREMADE_5x5',  # Ranked Premade 5v5 games
                    'RANKED_TEAM_5x5']     # Ranked Team 5v5 games

# Functions that check status of the data pulls
def isValid(match):
    if match.status_code != STATUS_OK:
        return False
    return True

# We only look at 'MATCHED_GAME' since 'CUSTOM_GAME' is where people try experimental champions and play styles
def isClassicMatch(match):
    j = match.json()
    if j['mapId'] == 11 and j['matchMode'] == "CLASSIC" and j['matchType'] == 'MATCHED_GAME':
        return True
    else:
        return False

# Checking if ranked game
def isRankMatch(match):
    j = match.json()
    if j['queueType'] in valid_rank_games:
        return True
    else:
        return False

# Check whether we went over the API Rate Limit                     
def rateLimitExceeded(match):
    if int(match.status_code) == STATUS_RATE_LIMIT_EXCEEDED:
        return True
    return False

### Pulling and Saving JSON Files <a id='JSON-Files'></a>
As mentioned above, matchID's do not have a defining pattern. For the most part, they seem to increase incrementally and similar game modes/types seem to be clustered around. Other times, they are long succession of empty numbers (404 status where there is no game stored for that matchID).

We start with a region (ex. North America) and a match ID from that region. We first have a list of matchID's of the world's high-ranked players' recent match history. 

In [5]:
# List of starting MatchID's for each region was found by looking up skilled players and their recent games
# on this website: http://www.lolsummoners.com/ladders/
regional_match_id = {'br': 667070831,
                     'eune': 1297476531,
                     'euw': 2416867378,
                     'kr': 2192854102,
                     'lan': 229958232,
                     'las': 273070421,
                     'na': 2029999649,
                     'oce': 113870290,
                     'ru': 84050671,
                     'tr': 325244412}

Then, we pull the matchID, examine it to see if it meets our criteria, and save it to as a JSON file if it does. Then, we increment the matchID and continue the process. Once we reach the rate limit for the region, we move onto the next region and continue the same process. Going through all 10 regions make it so that we are not limited by the rate limit. By the time we make a way around the world (all 10 regions), the rate limit time of 10 seconds for the first region has already passed. Thus, we do not have to waste any time waiting for the API to 'clear' us.

The data is saved in JSON format one by one in '/dump' sub-directory. Each JSON file is named by the region marker followed by the matchID.

Although it would be very unlikely given that matchID's are 10-digit numbers, we add a region marker in front of each matchID to make sure same matchID's from different regions do not overwrite one another.

In [42]:
# We modified the code from https://github.com/LionTurtle/TeamCompML/

# initializing indices
i, n, r = (0, 0, 0) # i = incrementing starting match ID, n = number of games retrieved, r = region list index)

# We keep the range low here for demonstration purposes. 
# In reality, we set the range to 100,000 and let it run days and nights
for k in range(100):
    
    # current region
    curr_region = REGION_list[r]
    
    # increment match ID of the current region
    current_match_id = regional_match_id[curr_region] + i
    i += 1
    
    # use the API function to receive information on this current_match_id minus timeline data
    m = get_match(curr_region, current_match_id, False)
    
    if not rateLimitExceeded(m):
        if isValid(m) and isClassicMatch(m) and isRankMatch(m):
            # we add curr_region marker in front of the current_match_id when naming the files
            f = open("dump_global/" + str(curr_region) + str(current_match_id) + ".json", 'w+')
            f.write(m.text.encode('utf-8'))
            n += 1
            print 'n: %d, ts: %s, ID: %d, date: %s, season: %s, tier: %s, loop_index: %d, region: %s' % (n, datetime.datetime.fromtimestamp(time.time()).strftime('%H:%M:%S'), 
                                                                                                 current_match_id, 
                                                                                                 ('%s' % (time.strftime('%m-%d', time.localtime(m.json()['matchCreation']/1000)))),
                                                                                                 m.json()['season'], m.json()['participants'][1]['highestAchievedSeasonTier'], k,
                                                                                                            m.json()['region'])
            f.close()
    else:  
        # we save the new match id (incremented match id) to pick up pulling data for when the next cycle comes around
        regional_match_id[curr_region] =  current_match_id + i
        
        # when we reach the rate limit for the region, we move onto the next region on the list
        if (r == 9):
            # go back to the beginning of the list of regions and start the cycle again
            print ("rate limit reached at %s, moving onto %s") % (REGION_list[r], REGION_list[0])
            r = 0
        else:
            print ("rate limit reached at %s, moving onto %s") % (REGION_list[r], REGION_list[r+1])
            r += 1
        # reset the incrementer    
        i = 0

n: 1, ts: 20:51:59, ID: 2029999649, date: 12-06, season: PRESEASON2016, tier: GOLD, loop_index: 0, region: NA
n: 2, ts: 20:52:00, ID: 2029999651, date: 12-06, season: PRESEASON2016, tier: UNRANKED, loop_index: 2, region: NA
n: 3, ts: 20:52:01, ID: 2029999652, date: 12-06, season: PRESEASON2016, tier: PLATINUM, loop_index: 3, region: NA
n: 4, ts: 20:52:03, ID: 2029999656, date: 12-06, season: PRESEASON2016, tier: GOLD, loop_index: 7, region: NA
rate limit reached at na, moving onto kr
n: 5, ts: 20:52:12, ID: 2192854110, date: 12-06, season: PRESEASON2016, tier: SILVER, loop_index: 19, region: KR
rate limit reached at kr, moving onto eune
n: 6, ts: 20:52:18, ID: 1297476540, date: 12-06, season: PRESEASON2016, tier: PLATINUM, loop_index: 31, region: EUNE
rate limit reached at eune, moving onto euw
rate limit reached at euw, moving onto lan
rate limit reached at lan, moving onto las
rate limit reached at las, moving onto oce
n: 7, ts: 20:52:37, ID: 113870297, date: 12-02, season: PRESEASON

### Pre-processing Data <a id='pre-processing'></a>

Let's take a look at what the JSON files look like. It has a bunch of metrics that we won't use. We will focus on the in-game stats broken down by participant (player) that give us a quantitative overview of the person's play style and performance. 

In [7]:
test_response = get_match('na', 2026610783, False).json()
# let's take a look at what it looks like
print test_response

{u'queueType': u'RANKED_TEAM_5x5', u'matchVersion': u'5.23.0.239', u'platformId': u'NA1', u'season': u'PRESEASON2016', u'region': u'NA', u'matchId': 2026610783, u'mapId': 11, u'matchCreation': 1449099999881, u'teams': [{u'firstDragon': False, u'bans': [{u'pickTurn': 1, u'championId': 203}, {u'pickTurn': 3, u'championId': 41}, {u'pickTurn': 5, u'championId': 420}], u'firstInhibitor': False, u'baronKills': 0, u'firstRiftHerald': False, u'winner': True, u'firstBaron': False, u'riftHeraldKills': 1, u'firstBlood': True, u'teamId': 100, u'firstTower': False, u'vilemawKills': 0, u'inhibitorKills': 0, u'towerKills': 6, u'dominionVictoryScore': 0, u'dragonKills': 1}, {u'firstDragon': True, u'bans': [{u'pickTurn': 2, u'championId': 117}, {u'pickTurn': 4, u'championId': 16}, {u'pickTurn': 6, u'championId': 223}], u'firstInhibitor': False, u'baronKills': 0, u'firstRiftHerald': True, u'winner': False, u'firstBaron': False, u'riftHeraldKills': 1, u'firstBlood': False, u'teamId': 200, u'firstTower': 

Oh, boy. That's a big jumble of text! 

Before we start making csv files of nicely organized data of the JSON files, we make a function that converts a championID to champion name. Our raw data denotes each champion by its championID, which is just a random number.

In [8]:
########## Champion ID to Champion Name converter ##########

# GET the static data on champion keys and names
static_champ_data = requests.get("https://global.api.pvp.net/api/lol/static-data/na/v1.2/champion?locale=en_US&api_key=%s"%KEY).json() 
champs = static_champ_data['data']
champ_names = static_champ_data['data'].keys()

# initializing index and empty dictionaries
i = 0
list_champs = []
list_champ_ids = []

# filling in the lists
for i in range(len(champ_names)):
    this_champ = champs[champ_names[i]]
    list_champs.append(this_champ['key'])
    list_champ_ids.append(this_champ['id'])
    
# champion ID and name dictionary    
champ_dict = dict(zip(list_champ_ids, list_champs))

# Wukong's name is not correct (MonkeyKing)
champ_dict[62] = 'Wukong'

# function that converts a champion ID to champion name
def champion_name(champion_id):
    if champion_id in champ_dict:
        return champ_dict[champion_id]
    else:
        return 'no_name'

In [9]:
# Let's checkout what champions 3 and 64 are
print champion_name(3)
print champion_name(64)

Galio
LeeSin


Now that we have a bunch of JSON files. Let's put them into usable form. We will merge all the necessary data from the JSON files to a giant csv file for easy access. Each game has 10 players (thus champions) and thus will have 10 rows of data. Each game and its participants are identifiable by the match ID. 

This part takes a few minutes to run depending on how much data we have on hand.

In [10]:
# Helper functions to parse data into a csv file
def insertComma(fileName):
    fileName.write(' , ')

def writetoFile(fileName,data):
    fileName.write(str(data))
    insertComma(fileName)

### Feature Selection<a id='feature-selection'></a>
The retrieved data gives us A LOT of statistics and metrics regarding the player and what happened in that game. However, we only want to work with the most important, dividing features that will facilitate the clustering process as well as CURSE OF DIMENSIONALITY.

The following list are the features we decided to keep and the reasoning behind each of them. Using our domain knowledge of the game, we believe that these features would be important in differentiating the roles of different champions once we get to clustering.

##### General Info for Labeling and Group-by Operations (not used for clustering later)
<b>match_ID</b>: unique match ID; tells us what game to which the statistics belong  
<b>participan</b>t: general participant identifier within the game (1~10)  
<b>champ</b>: name of the participant's champion    

##### Relevant In-game Stats
<b>win</b>: boolean flag for winning team and losing team (0,1)  
<b>kills</b>: number of kills the champion had during game    
<b>assists</b>: number of assist kills the champion had during game  
<b>deaths</b>: number of times the champion died in the game    
<b>gold_earned</b>: amount of gold earned throughout the game    
<b>p_damage_to_champs</b>: physical damage dealt to enemy champions (AD: attack damage)    
<b>m_damage_to_champs</b>: magic damage dealt to enemy champions (AP: ability power)  
<b>max_multi_kill</b>: the highest multi kill achieved (ex. 5 -> pentakill, where one champion has wiped out the entire enemy team)  
<b>max_crit</b>: highest critical physical damage dealt by the champion  
<b>damage_taken</b>: the amount of damage that the champion received  
<b>no_killing_spree</b>: number of consecutive kills before dying  
<b>creep_score</b>: number of minions killed  
<b>jungle_killed</b>: number of neutral jungle monsters killed  
<b>cc_dealt</b>: metric that tells how much crowd-control (cc) effect was dealt to enemy champions  
<b>wards_placed</b>: number of sight wards placed on the map  
<b>total_heal</b>: amount of healing done to me or teammates excluding lifesteal, natural health regeneration, and potions

We believe that these features are defining characteristics of champion roles. For example, an attack-damage (AD) based assassin/fighter such as Master Yi will have high <b>max_crit</b> and <b>max_multi_kill</b>. Since he does not have innate crowd-control abilities, he will have near 0 <b>cc_dealt</b> stat. Once we start clustering, we predict that he will be in a cluster with other similar champions who rely on short burst of physical attack damage.

To give another example of how these features may help, let's look back at the example of Malphite we saw above. Malphite, as mentioned, mostly functions as a tanker. He will thus be in the cluster characterized by high <b>damage-taken</b> and high <b>cc_dealt</b> given that he is designed to endure a lot of damage and has several crowd-control abilities.

### Parsing JSON file into a CSV file <a id='parsing'></a>

Now we go through the JSON files in our directory, pull out the relevant data, and save them into an aggregated csv file.

Each row represents a player's match record. Since there are 10 people in a match, each file we downloaded should give us 10 rows of data to work with.

This gives us one simple file we can rely on to play with!

In [35]:
# We modified the code from https://github.com/LionTurtle/TeamCompML/

# directory with all the JSON files
DUMP_DIR = 'dump_global/'
counter = 0
# the csv file where we are saving everything
unnormalized_data = open('unnormalized.csv','w+')
# column names (the features above)
unnormalized_data.write('region,match_ID,participant,rank,champ,win,kills,assists,deaths,gold_earned,p_damage_to_champs,m_damage_to_champs,max_multi_kill,max_crit,damage_taken,no_killing_spree,creep_score,jungle_killed,cc_dealt,wards_placed,total_heal\n')        

for f in os.listdir(DUMP_DIR):
    counter += 1
    if (counter % 1000) == 0:
        print "parsed %d files..." % counter
    
    if f.find('json') != -1:
        json_data = open(DUMP_DIR + f)
        data = json.load(json_data)
        json_data.close()

        if len(data['participants']) == 10:  #verify valid game with all players
            counter += 1

            for i in range(0,10):
                teamID = 0 if data['participants'][i]['teamId'] == data['teams'][0]['teamId'] else 1
                stats = data['participants'][i]['stats']

                # General Info -- not used for clustering
                writetoFile(unnormalized_data, data['region'])
                writetoFile(unnormalized_data, data['matchId'])
                writetoFile(unnormalized_data, data['participantIdentities'][i]['participantId'])
                writetoFile(unnormalized_data, data['participants'][i]['highestAchievedSeasonTier'])
                writetoFile(unnormalized_data, champ_dict[data['participants'][i]['championId']])
                
                # indicator for winner
                writetoFile(unnormalized_data, int(data['teams'][teamID]['winner']))
                
                # KDA
                writetoFile(unnormalized_data, stats['kills'])
                writetoFile(unnormalized_data, stats['assists'])
                writetoFile(unnormalized_data, stats['deaths'])
                
                writetoFile(unnormalized_data, stats['goldEarned'])
                writetoFile(unnormalized_data, stats['physicalDamageDealtToChampions'])
                writetoFile(unnormalized_data, stats['magicDamageDealtToChampions'])
                writetoFile(unnormalized_data, stats['largestMultiKill'])
                writetoFile(unnormalized_data, stats['largestCriticalStrike'])
                
                writetoFile(unnormalized_data, stats['totalDamageTaken'])
                writetoFile(unnormalized_data, stats['killingSprees'])
                
                writetoFile(unnormalized_data, stats['minionsKilled'])
                writetoFile(unnormalized_data, stats['neutralMinionsKilled'])
                writetoFile(unnormalized_data, stats['totalTimeCrowdControlDealt'])
                
                writetoFile(unnormalized_data, stats['wardsPlaced'])
                writetoFile(unnormalized_data, stats['totalHeal'])

                unnormalized_data.write('\n')

parsed 1000 files...
parsed 2000 files...
parsed 3000 files...
parsed 4000 files...
parsed 5000 files...
parsed 6000 files...
parsed 7000 files...
parsed 8000 files...
parsed 9000 files...
parsed 10000 files...
parsed 11000 files...
parsed 12000 files...
parsed 13000 files...
parsed 14000 files...
parsed 15000 files...
parsed 16000 files...
parsed 17000 files...
parsed 18000 files...
parsed 19000 files...
parsed 20000 files...
parsed 21000 files...
parsed 22000 files...
parsed 23000 files...
parsed 24000 files...
parsed 25000 files...
parsed 26000 files...
parsed 27000 files...
parsed 28000 files...


##### Putting the data in pandas dataframe

Now we use the csv file to make a pandas dataframe, a format we are familiar with!

In [30]:
# Read the csv file
df = pd.read_csv('unnormalized.csv')

# There's a weird column named 'Unnamed: 21'. Let's get rid of it
#del df['Unnamed: 21']

# check column names
df.columns

Index([u'region', u'match_ID', u'participant', u'rank', u'champ', u'win',
       u'kills', u'assists', u'deaths', u'gold_earned', u'p_damage_to_champs',
       u'm_damage_to_champs', u'max_multi_kill', u'max_crit', u'damage_taken',
       u'no_killing_spree', u'creep_score', u'jungle_killed', u'cc_dealt',
       u'wards_placed', u'total_heal'],
      dtype='object')

In [31]:
# Let's see what our data looks like
df.head(20)

,region,match_ID,participant,rank,champ,win,kills,assists,deaths,gold_earned,...,m_damage_to_champs,max_multi_kill,max_crit,damage_taken,no_killing_spree,creep_score,jungle_killed,cc_dealt,wards_placed,total_heal
NA,2026610783,1,DIAMOND,Karma,1,2,8,1,6865,1035,...,2,0,8073,1,20,0,163,16,1134,
NA,2026610783,2,DIAMOND,Caitlyn,1,2,3,3,8014,5585,...,1,1100,7410,0,136,4,20,6,1362,
NA,2026610783,3,PLATINUM,Orianna,1,3,7,0,8440,555,...,1,0,6848,1,137,5,804,6,1337,
NA,2026610783,4,DIAMOND,DrMundo,1,7,5,1,8974,3577,...,2,0,17087,1,20,61,553,14,8175,
NA,2026610783,5,PLATINUM,Jax,1,7,4,3,10090,7938,...,2,0,15120,3,133,10,34,6,3554,
NA,2026610783,6,DIAMOND,Janna,0,1,6,5,5634,1290,...,1,0,9664,0,12,0,94,12,2159,
NA,2026610783,7,DIAMOND,Jinx,0,1,5,3,6803,9711,...,1,1162,8234,0,130,3,87,7,1165,
NA,2026610783,8,UNRANKED,Elise,0,3,4,3,6961,1094,...,2,0,14033,1,35,41,315,10,3097,
NA,2026610783,9,UNRANKED,Olaf,0,2,1,8,6682,7772,...,1,0,15385,0,105,7,287,7,960,
NA,2026610783,10,DIAMOND,TwistedFate,0,1,1,2,6739,342,...,1,0,9442,0,110,10,96,12,472,


The data frame looks good! We now explore our data.

##Part II: Exploratory Data Analysis (EDA) <a id='EDA'></a>

### Rank Tier and Region Distribution<a id='tier-region-dist'></a>

We have compiled match data from various rank tiers and regions. Let's check out the distribution of those before we move on.

To give a little context, LoL has a <b>rank tier system.</b> When people play "ranked" games (which is what we specifically downloaded), they play with other people who are of similar playing level. Those who have not played enough ranked games are marked to be in "unranked" tier. Then, goes Bronze, Silver, Gold, Platinum, Diamind, Master, and Challenger in order of increasing skill level. The tier system is based on a percentile. The rank tiers are also broken down into divisions (ex. Bronze III, Platinum I, etc.), but we are not concerned about these subvisions for this project.

This helpful website (http://www.leagueofgraphs.com/rankings/rank-distribution) shows that the distribution of tiers are as follows:

Of those players who are considered "ranked,"  
<b>Bronze</b>: &nbsp;&nbsp;&nbsp; 56.5% of the population  
<b>Silver</b>: &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 32.5% of the population  
<b>Gold</b>: &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;9.1% of the population  
<b>Platinum</b>: &nbsp;&nbsp;&nbsp;&nbsp;1.6% of the population  
<b>Diamond</b>: &nbsp;&nbsp;&nbsp;&nbsp;0.3% of the population  
<b>Master</b>: &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;0.01% of the population  
<b>Challenger</b>: 0.0000....1% of the population (this is where professional gamers are)  

To interpret this more simply, Bronze players are in the bottom half in terms of playing abilities and Diamond players are in the top 0.3% percentile.

Let us check whether our pulled data reflects this distribution.

In [ ]:
# List of Regions
regions = map(lambda x:x.upper(), REGION_list)
print regions

# Rank Tier Breakdown
region_dist = dict(df['region'].value_counts()/10)
rank_dist = dict(df['rank'].value_counts())

print region_dist
print rank_dist

plt.bar(range(len(region_dist)), region_dist.values(), align='center')
plt.xticks(range(len(region_dist)), region_dist.keys())
plt.show()

#plt.show()
#fig, ax = plt.subplots()
#df['region'].value_counts().plot(ax=ax, kind='barh')
#plt.show()


In [104]:
load_my_data()

NameError: name 'load_my_data' is not defined

### Most Popular Champions

### Win/Loss Rate Based on Champion<a id='win_loss_rate'></a>

###KDA by Champion<a id='KDA'></a>

###Champions by Stat<a id='champion-stat'></a>